
# API Requests from Postman Collection
This notebook contains API requests extracted from the provided Postman collection.


In [ ]:

# 0. /SASLogon/oauth/token
import requests
import json
import urllib3
# Disable all warnings from urllib3
urllib3.disable_warnings()

sasserver = "https://your_server"
payload = {
    'grant_type': 'password',
    'username': 'your_username',
    'password': 'your_password'
}

url = f"{sasserver}/SASLogon/oauth/token"
headers = {'Authorization': 'Basic c2FzLmVjOg=='}
response = requests.post(url, headers=headers, data=payload,verify=False)
access_token = response.json()["access_token"]

print(json.dumps(response.json(), indent=4))

In [ ]:

# 1. Create Discovery Agent
url = f"{sasserver}/catalog/bots"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ' + access_token
}

payload = json.dumps({
    "provider": "TABLE-BOT",
    "name": "Crawl Samples Use Case (CAS)",
    "description": "Crawl the Samples caslib",
    "parameters": {
    	"datasourceURI": "/dataSources/providers/cas/sources/cas-shared-default~fs~Samples",
        "region": "US NC"
    }
})

response = requests.post(url, headers=headers, data=payload, verify=False)
botId = response.json()["id"]
print(json.dumps(response.json(), indent=4))

    

In [ ]:

# 2. Run Discovery Agent
url = f"{sasserver}/catalog/bots/{botId}/state?value=running"
headers = {
  'Accept': 'text/plain',
  'Authorization': 'Bearer ' + access_token
}

response = requests.put(url, headers=headers, data=None, verify=False)
print(response.text)
    

In [ ]:

# 3. Discovery Agent state
import time
url = f"{sasserver}/catalog/bots/{botId}/state"
response = requests.get(url, headers=headers, data=None, verify=False)
start_time = time.time()

while response.text != 'idle':
    response = requests.get(url, headers=headers, data=None, verify=False)
    print(response.text)
    time.sleep(5)
end_time = time.time()

print(f"Agent took {round(end_time - start_time)}s to run")
    

In [ ]:

# 4. Discovery Agent
url = f"{sasserver}/catalog/bots/{botId}"
headers = {
    'Accept': 'application/vnd.sas.metadata.bot.summary+json',
    'Authorization': 'Bearer ' + access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(json.dumps(response.json(), indent=4))
    

In [ ]:

# 5. Download dataDictionary filtered by name with prefix
url = f"{sasserver}/catalog/instances?filter=contains(name,COST)&level=dataDictionary&prefix=simpledownload"
headers = {
    'Accept': 'text/csv',
    'Authorization': 'Bearer ' + access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(response.text)    

In [ ]:

# 6. Download dataDictionary, multiple tables filtered by name with prefix
url = f"{sasserver}/catalog/instances/?filter=or(contains(name,COST),contains(name,WARRANTY))&level=dataDictionary&prefix=multipletables"
headers = {
    'Accept': 'text/csv',
    'Authorization': 'Bearer ' + access_token
}
response = requests.get(url, headers=headers, data=None, verify=False)
print(response.text)   

In [ ]:
# 7. Download dataDictionary, filtered by name, prefix and dateTimeStampSuffix set to false

url = f"{sasserver}/catalog/instances/?filter=or(contains(name,COST),contains(name,WARRANTY))&level=dataDictionary&prefix=multipletables_nodatetimesuffix&dateTimeStampSuffix=false"

headers = {'Accept': 'text/csv',  'Authorization': 'Bearer ' + access_token}
response = requests.get(url, headers=headers,data=None, verify=False)
print(response.status_code)
print(response.text)

In [ ]:
# 8. Upload dataDictionary to CAS filtered by name with prefix

url = f"{sasserver}/catalog/instances/?filter=contains(name,COST)"

headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/vnd.sas.metadata.instance.upload.request+json', 'Accept': 'application/vnd.sas.metadata.instance.upload.request+json'}
# Replace placeholders in the body with actual values
data = '''{
    "level": "dataDictionary",
    "prefix": "dataDictionary_cost_table_upload",
    "dateTimeStampSuffix": true,
    "serverName": "cas-shared-default",
    "caslibName": "Public"
}'''
response = requests.post(url, headers=headers, data=data,verify=False)
print(response.status_code)
print(response.text)


In [ ]:
# 9. Download detailedMetrics filtered by name with prefix

url = f"{sasserver}/catalog/instances/?filter=contains(name,COST)&level=detailedMetrics&prefix=detailedmetrics_costTable"

headers = {'Accept': 'text/csv','Authorization': 'Bearer ' + access_token}
response = requests.get(url, headers=headers,verify=False)
print(response.status_code)
print(response.text)


In [ ]:
# 10. Upload detailedMetrics to CAS filtered by name with prefix
# Replace placeholders in the URL with actual values
url = f"{sasserver}/catalog/instances/?filter=contains(name,COST)"
# Replace placeholders in the headers with actual values
headers = {  'Authorization': 'Bearer ' + access_token,'Content-Type': 'application/vnd.sas.metadata.instance.upload.request+json', 'Accept': 'application/vnd.sas.metadata.instance.upload.request+json'}
# Replace placeholders in the body with actual values
data = '''{
    "level": "detailedMetrics",
    "prefix": "detailedMetrics_cost_table_upload",
    "dateTimeStampSuffix": true,
    "serverName": "cas-shared-default",
    "caslibName": "Public"
}'''
response = requests.post(url, headers=headers, data=data,verify=False)
print(response.status_code)
print(response.text)

In [ ]:
# 11. Download profile Metrics filtered by name with prefix
# Replace placeholders in the URL with actual values
url = f"{sasserver}/catalog/instances?filter=contains(name,COST)&level=dataDictionaryAndProfile"
# Replace placeholders in the headers with actual values
headers = {'Authorization': 'Bearer ' + access_token, 'Content-Type': 'application/vnd.sas.metadata.instance.upload.request+json', 'Accept': 'text/csv'}
response = requests.get(url, headers=headers,verify=False)
print(response.status_code)
print(response.text)

In [ ]:
# 12. Upload profileMetrics to CAS with filter all data in the Samples discovery agent crawled library
# Replace placeholders in the URL with actual values
url = f"{sasserver}/catalog/instances?filter=contains(resourceId,'cas~fs~cas-shared-default~fs~Samples')"
# Replace placeholders in the headers with actual values
headers = {  'Authorization': 'Bearer ' + access_token,'Content-Type': 'application/vnd.sas.metadata.instance.upload.request+json', 'Accept': 'application/vnd.sas.metadata.instance.upload.request+json'}
# Replace placeholders in the body with actual values
data = '''{
    "level": "dataDictionaryAndProfile",
    "prefix": "dataDictionaryAndProfile_all_Samples_table_upload",
    "dateTimeStampSuffix": true,
    "serverName": "cas-shared-default",
    "caslibName": "Public"
}'''
response = requests.post(url, headers=headers, data=data, verify=False)
print(response.status_code)
print(response.text)

In [ ]:

# 13. Delete Discovery Agent
url = f"{sasserver}/catalog/bots/{botId}"
headers = {
    'Authorization': 'Bearer ' + access_token
}
response = requests.delete(url, headers=headers, data=None, verify=False)
print(response.text)
    